In [1]:
import torch, torch.onnx, tqdm, time, os, json, multiprocessing
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from models.est_model import ResidualRegression, DNNRegression
from torch.utils.data import DataLoader, Dataset, random_split
from pytorch_lightning.callbacks import DeviceStatsMonitor, EarlyStopping
from sklearn.metrics import r2_score

D:\Workspace\Python\estimate_crawler_crane_ground_pressure\venv\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
monitor = DeviceStatsMonitor()

In [4]:
# set train setting
save_model = True
train_num_epoch = 50000
min_loss = 100
dl_workers = 0

In [5]:
gpu = torch.device('cuda')
torch.set_float32_matmul_precision('high')

In [6]:
# load hyperparameter of json file
with open('.' + os.sep + os.path.join('models', 'params_dnn_20220207-012403.json'), 'r') as file:
    hyper_params = json.load(file)

n_inputs = hyper_params['n_of_inputs']
n_outputs = hyper_params['n_of_outputs']
n_layers = hyper_params['n_of_hidden']

In [7]:
model = ResidualRegression(n_inputs, n_layers, n_outputs)
print(model)

ResidualRegression(
  (input_layer): Sequential(
    (0): Linear(in_features=4, out_features=88, bias=False)
  )
  (output_layer): Sequential(
    (0): Linear(in_features=88, out_features=22, bias=False)
  )
  (res_block_1): ResidualBlock(
    (active): ReLU()
    (layers): Sequential(
      (0): Linear(in_features=88, out_features=88, bias=True)
      (1): ReLU()
      (2): Linear(in_features=88, out_features=88, bias=True)
      (3): ReLU()
      (4): Linear(in_features=88, out_features=88, bias=True)
      (5): ReLU()
      (6): Linear(in_features=88, out_features=88, bias=True)
      (7): ReLU()
      (8): Linear(in_features=88, out_features=88, bias=True)
      (9): ReLU()
      (10): Linear(in_features=88, out_features=88, bias=True)
      (11): ReLU()
      (12): Linear(in_features=88, out_features=88, bias=True)
      (13): ReLU()
      (14): Linear(in_features=88, out_features=88, bias=True)
      (15): ReLU()
      (16): Linear(in_features=88, out_features=88, bias=True)
    

In [8]:
data = pd.read_csv('./resources/sim_data_edit.csv')
feature_names = ['lift_weight(ton)', 'lift_height(m)', 'rising_angle(deg)', 'swing_angle(deg)']
# target_names = ['left_ground_pressure_min(kg/cm2)', 'left_ground_pressure_max(kg/cm2)', 'left_pressure_length(m)', 'right_ground_pressure_min(kg/cm2)', 'right_ground_pressure_max(kg/cm2)', 'right_pressure_length(m)']
left_target_names = ['left-0.0m', 'left-0.675m', 'left-1.35m', 'left-2.025m', 'left-2.7m', 'left-3.375m', 'left-4.05m', 'left-4.725m', 'left-5.4m', 'left-6.075m', 'left-6.75m']
right_target_names = ['right-0.0m', 'right-0.675m', 'right-1.35m', 'right-2.025m', 'right-2.7m', 'right-3.375m', 'right-4.05m', 'right-4.725m', 'right-5.4m', 'right-6.075m', 'right-6.75m']
target_names = left_target_names + right_target_names

feature_data = []
target_data = []

for feature_name in feature_names:
    feature_data.append(data[feature_name])

for target_name in target_names:
    target_data.append(data[target_name])

feature_data = np.array(feature_data, dtype=np.float32).T
target_data = np.array(target_data, dtype=np.float32).T

class TensorDataset(Dataset):
    def __init__(self, feature, target):
        self.x_data = torch.tensor(feature)
        self.y_data = torch.tensor(target)
        self.len = self.x_data.shape[0]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

train_dataset = TensorDataset(feature_data, target_data)
train_data_loader = DataLoader(train_dataset, batch_size=len(train_dataset), num_workers=multiprocessing.cpu_count())

In [9]:
# train model
early_stop_callback = EarlyStopping(monitor='train_loss', mode='min', verbose=True, min_delta=0.001, patience=200)
trainer = pl.Trainer(callbacks=[early_stop_callback], accelerator='cpu', enable_progress_bar=True, max_epochs=10000)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
D:\Workspace\Python\estimate_crawler_crane_ground_pressure\venv\lib\site-packages\pytorch_lightning\trainer\setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
D:\Workspace\Python\estimate_crawler_crane_ground_pressure\venv\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [10]:
trainer.fit(model=model, train_dataloaders=train_data_loader)


  | Name         | Type          | Params
-----------------------------------------------
0 | input_layer  | Sequential    | 352   
1 | output_layer | Sequential    | 1.9 K 
2 | res_block_1  | ResidualBlock | 78.3 K
3 | res_block_2  | ResidualBlock | 78.3 K
4 | res_block_3  | ResidualBlock | 78.3 K
5 | res_block_4  | ResidualBlock | 78.3 K
6 | res_block_5  | ResidualBlock | 78.3 K
7 | loss         | MSELoss       | 0     
-----------------------------------------------
393 K     Trainable params
0         Non-trainable params
393 K     Total params
1.576     Total estimated model params size (MB)
D:\Workspace\Python\estimate_crawler_crane_ground_pressure\venv\lib\site-packages\pytorch_lightning\loops\fit_loop.py:281: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

RuntimeError: DataLoader worker (pid(s) 37800, 37820, 37840, 35936, 37260, 23892, 36812, 37272, 37576, 34300, 30168, 36764, 2060, 32736, 36476, 29216, 26632, 32804, 36528, 34432, 12564) exited unexpectedly

In [10]:
torch.save(model.state_dict(), './models/est_ground_pressure.pt')

In [11]:
model.eval()
for data in train_dataset:
    print(r2_score(data[1].detach().numpy() ,model(data[0]).detach().numpy()))
    print(model(data[0]).detach().numpy())
    print(data[1].detach().numpy())

0.9991166927100125
[ 3.9801526   3.264197    2.5735798   1.8104215   1.0852942   0.34398705
 -0.03680294 -0.03060333  0.01753594  0.01136806  0.05508342  3.908132
  3.1767843   2.4969532   1.8165781   1.12157     0.2996784  -0.05725452
 -0.07357418 -0.01126215  0.0543755   0.04786623]
[3.97  3.25  2.529 1.809 1.089 0.368 0.    0.    0.    0.    0.    3.97
 3.25  2.529 1.809 1.089 0.368 0.    0.    0.    0.    0.   ]
0.9991550326938547
[ 2.5656154e+00  2.1667466e+00  1.7175603e+00  1.3361852e+00
  8.9341944e-01  3.9287645e-01 -2.6940152e-02  1.7139316e-03
 -3.1142935e-02 -6.8648644e-03  1.6497225e-03  4.6124811e+00
  3.8011262e+00  3.0402205e+00  2.3046646e+00  1.5583057e+00
  7.3223573e-01  1.6591266e-01  1.5040267e-02 -1.2192786e-02
 -2.4368256e-02  1.5258983e-02]
[2.61  2.18  1.751 1.321 0.891 0.462 0.032 0.    0.    0.    0.    4.58
 3.826 3.072 2.318 1.564 0.81  0.056 0.    0.    0.    0.   ]
0.9988705881047041
[ 1.1640627   0.9809619   0.9014899   0.7045685   0.548369    0.4601706

In [12]:
torch.onnx.export(model, torch.zeros(4), './models/est_ground_pressure.onnx')

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================
